# 5 Training ML models with Serotiny
**Estimated time to run through notebook is 20 minutes** 

This notebook shows how to
-  [Load libraries, predefine some functions, and load the manifest ](#preprocessing) 
-  [5.1 Parametrize a ML task using serotiny's yamls](#param)
-  [5.2 Train a classification model based on 2D images](#train)
-  [5.3 Load and apply a trained model](#apply)
-  [5.4 Train a classification model based on 3D images](#train3D)
-  [Conclusion](#end)

#### Resources 
- Serotiny code: https://github.com/AllenCell/serotiny
- Serotiny documentation: https://allencell.github.io/serotiny
- Hydra for configurability https://hydra.cc/
- MLFlow for experiment tracking https://mlflow.org/
- Pytorch Lightning for DL training/testing/predictions https://pytorchlightning.ai/

## <a id='preprocessing'></a>Load libraries, predefine some functions, and load the manifest 


### Load libraries and predefined functions

In [ ]:
from upath import UPath as Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nbvv

from serotiny.io.image import image_loader
from cytodata_aics.io_utils import rescale_image

### Load the manifest and explore dimensions

In [ ]:
df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
print(f'Number of cells: {len(df)}')
print(f'Number of columns: {len(df.columns)}')

## <a id='param'></a>5.1 Parametrize a ML task using serotiny's yamls

`serotiny` is a Python package and framework to help you create configurable and reproducible DL projects. It uses [hydra](https://hydra.cc/) for configurability, [MLFlow](https://mlflow.org/) for experiment tracking,
and [Pytorch Lightning](https://pytorchlightning.ai/) for DL model training/testing/predictions.

### Project structure
With `serotiny` a DL project has a predefined structure (which this repo already complies with). To start a new project with the appropriate structure, you can use the [serotiny-project cookiecutter](https://github.com/allencellmodeling/serotiny-project-cookiecutter)

A serotiny project contains a Python package, and a config folder. This config folder is composed of 5 config groups:

<img src="resources/serotiny.png" width="700"/>


### `serotiny` commands
Aside from the predefined structure and config folder, `serotiny` has set of commands which know how to read a project's configuration (and override it)
and execute DL tasks.

For example, we could train a model using the a model config called `my_classifier` (which would live in `config/model/my_classifier.yaml`), and a data config
called `my_train_data` (which would live in `config/data/my_train_data.yaml`) and overriding some of the `mlflow` config parameters.
<br><small>Note: Because we didn't specify a top-level `mlflow` config, i.e. we didn't do `mlflow=...`, `serotiny` will use the default config, which lives in `config/mlflow/default.yaml`</small>

```
$ serotiny train model=my_classifier data=my_train_data mlflow.experiment_name=some_experiment mlflow.run_name=1st_run
```

Once the model finishes training, we could use it to make predictions on a different dataset, configured in `my_predict_data`

```
$ serotiny predict model=my_classifier data=my_predict_data mlflow.experiment_name=some_experiment mlflow.run_name=1st_run
```


## <a id='train'></a>5.2 Train a classification model based on 2D images


### Make a simple dataset of edge vs. non-edge cells

In [ ]:
from serotiny.transforms.dataframe.transforms import split_dataframe

Path("/home/aicsuser/serotiny_data/").mkdir(parents=True, exist_ok=True)

n = 1000 # number of cells per class
cells_edgeVSnoedge = df.groupby("edge_flag").sample(n)

# Add the train, test and validate split
cells_edgeVSnoedge = split_dataframe(dataframe=cells_edgeVSnoedge, train_frac=0.7, val_frac=0.2, return_splits=False)

cells_edgeVSnoedge.to_csv("/home/aicsuser/serotiny_data/cells_edgeVSnoedge.csv") 
print(f"Number of cells: {len(cells_edgeVSnoedge)}")
print(f"Number of columns: {len(cells_edgeVSnoedge.columns)}")

### Parametrize the data and model configurations

#### `data` config

```yaml
_target_: serotiny.datamodules.ManifestDatamodule

path: /home/aicsuser/serotiny_data/cells_edgeVSnoedge.csv

batch_size: 64
num_workers: 1
loaders:
  id:
    _target_: serotiny.io.dataframe.loaders.LoadColumn
    column: CellId
    dtype: int
  class:
    _target_: serotiny.io.dataframe.loaders.LoadColumn
    column: edge_flag
    dtype: float32
  image:
    _target_: serotiny.io.dataframe.loaders.LoadImage
    column: max_projection_z
    select_channels: ['membrane']  
    
split_column: "split"
```

#### `model` config

```yaml
_target_: serotiny.models.BasicModel
x_label: image
y_label: class
network:
  _target_: torch.nn.Sequential
  _args_:
    # conv block 1
    - _target_: torch.nn.LazyConv2d
      out_channels: 4
      kernel_size: 3
      stride: 1
    - _target_: torch.nn.LeakyReLU
    - _target_: torch.nn.LazyBatchNorm2d

    # conv block 2
    - _target_: torch.nn.LazyConv2d
      out_channels: 4
      kernel_size: 3
      stride: 1
    - _target_: torch.nn.LeakyReLU
    - _target_: torch.nn.LazyBatchNorm2d
    
    # conv block 3
    - _target_: torch.nn.LazyConv2d
      out_channels: 4
      kernel_size: 3
      stride: 1
    - _target_: torch.nn.LeakyReLU
    - _target_: torch.nn.LazyBatchNorm2d

    # flatten and feed through linear layer
    - _target_: serotiny.networks.layers.Flatten
    - _target_: torch.nn.LazyLinear
      out_features: 1
    - _target_: torch.nn.Sigmoid
    
loss:
  _target_: torch.nn.BCELoss
  
# a function used by `serotiny predict` to store the results of feeding data through the model
save_predictions:
  _target_: cytodata_aics.model_utils.save_predictions_classifier
  _partial_: true

# fields to include in the output for each batch
fields_to_log:
  - id
 
```

#### `trainer`, `trainer/callbacks` and `mlflow`

We provided sensible defaults to these config sections, but invite and recommend you to take a look at them and change them as you see fit
(in `/home/aicsuser/cytodata-hackathon-base/cytodata_aics/config/...`)

#### Changing the working directory

In [ ]:
# we need the commands we type to be ran from the serotiny project root
# (because that's what `serotiny` expects) so we change directories here,
# so we can run commands within the notebook
import os
os.chdir("/home/aicsuser/cytodata-hackathon-base")

#### Creating a run name based on the current date and time

In [ ]:
from datetime import datetime

# util to avoid referring to the same run unintentionally
now_str = lambda : datetime.now().strftime("%Y%m%d_%H%M%S")

#### Starting a training. Track the training at http://mlflow.cytodata.allencell.org/

In [ ]:
run_name = f"theo_bfdna_xproj_{now_str()}"
print(run_name)

!serotiny train \
    model=example_classifier_2d \
    data=example_dataloader_2d \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    trainer.gpus=[0] \
    trainer.max_epochs=2

## <a id='apply'></a>5.3 Load and apply a trained model

### Make predictions based on the model we just trained

In [ ]:
!serotiny predict \
    model=example_classifier_2d \
    data=example_dataloader_2d \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    trainer.gpus=[0]

### Retrieving predictions from MLFlow

In [ ]:
import mlflow
from serotiny.ml_ops.mlflow_utils import download_artifact

In [ ]:
mlflow.set_tracking_uri("http://mlflow.mlflow.svc.cluster.local")

with download_artifact("predictions/model_predictions.csv", experiment_name="cytodata_chapter5", run_name=run_name) as path:
    predictions_2d_df = pd.read_csv(path)
    

In [ ]:
predictions_2d_df = predictions_2d_df.merge(cells_edgeVSnoedge[['CellId','split']].rename(columns={'CellId':'id'}), on = 'id')
predictions_2d_df

### Distribution of the continuous class predictions

In [ ]:
plt.hist(predictions_2d_df.yhat.to_numpy())
plt.show()

### Confusion matrices of train, valid and test splits 

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

# make confusion matrix for each split
splits = ['train','valid','test']
fig, axes = plt.subplots(nrows=1,ncols=len(splits),figsize=(10, 3), dpi=100)

for i,split in enumerate(splits):
    
    y_true = predictions_2d_df[predictions_2d_df['split']==split]['y'].to_numpy()
    y_pred = predictions_2d_df[predictions_2d_df['split']==split]['yhat'].to_numpy()
    y_pred = np.round(y_pred) #get to crisp binary class labels from posterior probability

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)    
    score = accuracy_score(y_true,y_pred) #compute accuracy score
    cm_df = pd.DataFrame(cm)
    sns.heatmap(cm_df, annot=True, fmt='d',ax = axes[i])
    axes[i].set_title(f'Accuracy on {split} is {score:.2f}')
    axes[i].set_xlabel('True')
    axes[i].set_ylabel('Predicted')

plt.show()



## <a id='train3D'></a> 5.4 Train a classification model based on 3D images

### Configure the 5 yamls and run the training

In [ ]:
run_name = f"some_3d_run_{now_str()}"

!serotiny train \
    model=example_classifier_3d \
    data=example_dataloader_3d \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    ++data.loaders.image.unsqueeze_first_dim=True \
    ++data.loaders.image.ome_zarr_level=1 \
    trainer.gpus=[0] \
    trainer.max_epochs=1 

Note: The above task train at 5871MiB / 16160MiB when using the image.ome_zarr_level=1 

### Make predictions from the pretrained model

In [ ]:
!serotiny predict \
    model=example_classifier_3d \
    data=example_dataloader_3d \
    mlflow.experiment_name=cytodata_chapter5 \
    mlflow.run_name={run_name} \
    ++data.loaders.image.unsqueeze_first_dim=True \
    ++data.loaders.image.ome_zarr_level=1 \
    trainer.gpus=[0]

### Retrieving predictions from MLFlow

In [ ]:
mlflow.set_tracking_uri("http://mlflow.mlflow.svc.cluster.local")

with download_artifact("predictions/model_predictions.csv", experiment_name="cytodata_chapter5", run_name=run_name) as path:
    predictions_3d_df = pd.read_csv(path)

In [ ]:
predictions_3d_df = predictions_3d_df.merge(cells_edgeVSnoedge[['CellId','split']].rename(columns={'CellId':'id'}), on = 'id')
predictions_3d_df
# print(len(predictions_3d_df))

In [ ]:
plt.hist(predictions_3d_df.yhat.to_numpy())
plt.show()

### Confusion matrices of train, valid and test splits 

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

# make confusion matrix for each split
splits = ['train','valid','test']
fig, axes = plt.subplots(nrows=1,ncols=len(splits),figsize=(10, 3), dpi=100)

for i,split in enumerate(splits):
    
    y_true = predictions_3d_df[predictions_3d_df['split']==split]['y'].to_numpy()
    y_pred = predictions_3d_df[predictions_3d_df['split']==split]['yhat'].to_numpy()
    y_pred = np.round(y_pred) #get to crisp binary class labels from posterior probability

    # Computer confusion matrix
    cm = confusion_matrix(y_true, y_pred)    
    score = accuracy_score(y_true,y_pred) #compute accuracy score
    cm_df = pd.DataFrame(cm)
    sns.heatmap(cm_df, annot=True, fmt='d',ax = axes[i])
    axes[i].set_title(f'Accuracy on {split} is {score:.2f}')
    axes[i].set_xlabel('True')
    axes[i].set_ylabel('Predicted')

plt.show()

# <a id='end'></a>Conclusion
In this chapter you learned how to parametrize ML models using serotiny. We trained 2D and 3D models to distinguish edge from non-edge cells. In the next chapter you will learn what the hackathon tasks are. The data and tools that you have explored in this and previous chapters will be the basis for understanding and solving the hackathon tasks.